# The Key-Value Secret Engine

This is the most commonly used secret engine in HashiCorp Vault. It enables the storage of arbitrary static blobs of data. It can thus be seen as a sort of password manager, except that it can be accessed and managed fully programatically.

## Use Cases

Typical use cases for the KV (Key-Value) secret engine are:

- Centralized password manager for a team. This is especially interesting if the team already has Vault in place and wants to take advantages of operational synergies (backups, versioning, etc).
- Distribution of secrets for systems that are not integrated into Vault, nor with an IdP. For instance you might have an external system that does not support an integration with Vault or your standard identity provider. You will thus not use any form of dynamic credential system, but instead store the credential in a Vault KV engine. From there, the credential can be distributed to the application that requires it at runtime via the wide range of integrations supported by Vault.

### Starting Vault

Let us start a Vault server. This will run Vault in the background and push the logs to `/tmp/vault.log`. If at any point in time the Vault crashes, this command will need to be used again to re-launch the Vault server.

For now, do not worry about the configuration with which we are starting the server. This will be covered in a separate module.

In [ ]:
nohup bash -c '
  vault server -dev -dev-root-token-id=root-token
' > /tmp/vault.log 2>&1 &
echo $! > /tmp/vault.pid

### Login to Vault

Let us login to Vault. We started a development version vor simplicity.

In [ ]:
export VAULT_ADDR="http://127.0.0.1:8200"
vault login root-token

### Enabling the KV Secrets Engine

As with all operations with Vault, enabling the KV secrets engine is done via the API. The CLI can be used to avoid having to call the API directly and provides a nicer way to interact with the API. Some operations, enabling secrets engines included, can also be performed over the web UI. **This is not the case for all operations**. Enabling a secrets engine via the CLI is extremely simple:

```bash
vault secrets enable -path=/my-secret-store/ -version=2 kv
```

This would enable a KV secrets engine (version 2, such that secrets versioning and more features are supported) on the `/my-secret-store/` path of the API.

Let us instead do this via the UI:

1. Navigate to `http://localhost:8200` in your browser.
2. Login using `root-token` as the token.
3. Click on "Secrets Engines" and then "Enable new engine +":
   ![](./assets/img/enable-secrets-engine.png)
4. Choose "KV".
5. Enter the path under which we want to publish the engine as "my-secret-store" and create the engine. You can also have a look at the advanced configuration. The version 2 is the default, hence why we do not provide it here explicitly:
   ![](./assets/img/kv-creation-screen.png)

Once you have performed this, you will be in the secret engine, and see that there are not secrets in there yet. Enough UI, let us get back to the CLI:

In [ ]:
# list the secrets engines, check that our new one is there
vault secrets list

Note that there are quite a few other secrets engines already there. The following engines are "system engines" and will always be present:

- `sys`: system endpoints to administer Vault.
- `identity`: an identity store to enable identity mappings within Vault.

The `cubbyhole` engine is very often there as well. It is a per token temporary KV engine. This means that this data is persisted there for your token. As soon as your token expires, the data in there will be wiped. It can be used for temporary work, but in practice is not often used. This can be disabled by policies such that tokens do not get access to their cubbyhole engine.

The `secret` engine is another KV engine. This is setup automatically in development setups by Vault. In practice it would not be there for "normal" deployments. Let us remove it:

In [ ]:
vault secrets disable secret/

### Writing and Reading Secrets

Now let us populate the `my-secret-store` engine with secrets. This can be done very easily with the `vault kv` subcommand. We will need to provide the command with the engine we want to talk to using the `-mount=my-secret-store` flag, and then `put <name> <field1>=<value1>` to create a secret. Note that you can provide several fields.

Create two secrets in your engine:

1. One with name `database` with fields `host`, `username`, and `password`. Use dummy strings for the values.
2. One with name `teams/cloud-operations/app/portal-frontend/stage/prod/api-token` with field `token` and value `test`.

In [ ]:
# show help to see how to use this
vault kv put -mount=my-secret-store -h

In [ ]:
vault kv put -mount=my-secret-store ...

### Explore using the UI

Use the UI to explore the secrets engine. You will see that secrets that have slashes in their name are handled like a path in Vault. This means that you can create structures in your Vault, such as we did for the second secret. Having a good structure is critical for permission management. For more information on this, check the module on policies.

1. Go on the UI and check out the secrets.
2. Investigate the information that is available to you, such as secret metadata, version history, etc.

### Overwrite a Secret

Let us pretend that the API token for the portal frontend changed. Its value is now `some-amazing-token`. Use the same command as before to update the secret. Note that the reply tells you that you have written a new version of the secret: version 2.

In [ ]:
vault kv put -mount=my-secret-store ...

### Check out the new Version in the UI

Go to the credential, and check out the new version:

![](./assets/img/kv-version-history.png)

In [ ]:
# you can also check the version history using the CLI
vault kv metadata get -mount=my-secret-store teams/cloud-operations/app/portal-frontend/stage/prod/api-token

### Rolling Back Credentials

Oh no, turns out `my-amazing-token` is not actually a new token for the portal frontend. We need the old token! We can easily rollback to older versions of the credential. Note that this is not an operation that is natively supported in the web UI (yet). On the CLI, you can however do this very simply using the `rollback` subcommand.

Rollback to version 1.

> **NOTE:** flags and options need to be provide before arguments on the CLI. Therefore `vault kv rollback -version=15 my-secret` will correctly interpret the version option, while `vault kv rollback my-secret -version=15` will not.

In [ ]:
vault kv rollback -mount=my-secret-store

### Inspect the Versions

Inspect the version history now (via CLI or UI). Note that rolling back to an old version creates a new version with only the value of the version you rolled back to. This is very important to have consistent history of your versions.

In [ ]:
# use the CLI or UI to inspect the version history

### Deleting Secrets

Ok, let us delete the secret. It is no longer needed. This can easily be done with the `delete` subcommand.

Delete the portal-frontend token secret.

In [ ]:
vault kv delete -mount=my-secret-store ...

### Check out the Versions Again

Check out the versions again. Note that we in fact only deleted the latest version (version 3), but that this is still visible, both in the UI and in the CLI.

```
$ # we can still find the secret in the store
$ vault kv list -mount=my-secret-store teams/cloud-operations/app/portal-frontend/stage/prod/
Keys
----
api-token

$ # we can no longer read it though
$ vault kv get -mount=my-secret-store teams/cloud-operations/app/portal-frontend/stage/prod/
No value found at my-secret-store/data/teams/cloud-operations/app/portal-frontend/stage/prod
```

This is because any delete is only a soft delete. This means we can `undelete` secrets, or roll them back to previous versions to restore these. In order to properly and permanently delete secrets, we need to `destroy` them. This can be done to either one or several versions of a secret. Let us destroy only version 1 of this secret.

If you want to detroy all versions of a secret, you will need to delete its metadata. This can be done using `metadata delete`. Let us use that to delete the `database` secret.

1. Use the `destroy` subcommand with option `-versions` to delete specifically version 1 of the portal-frontend credential.
2. Use the `metadata delete` subcommand to destroy all versions of the `database` secret.

In [ ]:
vault kv destroy -mount=my-secret-store ...

In [ ]:
vault metadata delete -mount=my-secret-store ...

> **NOTE:** if the UI still shows a secret after its metadata has been deleted, this is typically caused by browser caching. In such a case reload the page. You should see that the secret no longer exists.

Check out that only the version 1 of the portal frontend token was deleted. If you try to view the version, you will see that there is no more data there ...

![](./assets/img/kv-version-delete.png)

## Cleaning Up

At the end of each module, you should clean up your Vault instance. This is done by shutting it down and wiping its database to restore its state.

In [ ]:
kill $(cat /tmp/vault.pid)
rm /tmp/vault.log
rm /tmp/vault.pid